This notebook shows how to use NVIDIA NeMo (https://github.com/NVIDIA/NeMo) to construct a toy demo which will swap a voice in the audio fragment with a computer generated one.

At its core the demo does:

Automatic speech recognition of what is said in the file. E.g. converting audio to text
Adding punctuation and capitalization to the text
Generating spectrogram from resulting text
Generating waveform audio from the spectrogram.

In [1]:
%%bash
apt-get update && apt-get install -y libsndfile1 ffmpeg
pip install Cython
pip install nemo_toolkit['all']==1.0.0rc1


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: botocore 1.20.84 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
ERROR: sphinxcontrib-bibtex 2.2.1 has requirement Sphinx>=2.1, but you'll have sphinx 1.8.5 which is incompatible.


In [2]:
import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
import nemo.collections.tts as nemo_tts
import IPython

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...


[NeMo W 2021-05-30 20:44:09 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2021-05-30 20:44:09 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for t

[nltk_data]   Unzipping corpora/cmudict.zip.


In [3]:

# Download audio sample which we'll try
# This is a sample from LibriSpeech Dev Clean dataset - the model hasn't seen it before
Audio_sample = '2086-149220-0033.wav'
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
# Listen to it
IPython.display.Audio(Audio_sample)

--2021-05-30 20:44:11--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 52.219.99.82
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|52.219.99.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K   940KB/s    in 0.2s    

2021-05-30 20:44:12 (940 KB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



**Instantiate pre-trained NeMo models which we'll use**

from_pretrained(...) API downloads and initialized model directly from the cloud.

We will load audio_sample and convert it to text with QuartzNet ASR model (an action called transcribe). To convert text back to audio, we actually need to generate spectrogram with Tacotron2 first and then convert it to actual audio signal using WaveGlow vocoder.

In [4]:
# Speech Recognition model - QuartzNet
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5").cuda()
# Punctuation and capitalization model
punctuation = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained(model_name='punctuation_en_distilbert').cuda()
# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="tts_en_tacotron2").cuda()
# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="tts_waveglow_88m").cuda()

[NeMo I 2021-05-30 20:44:17 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_quartznet15x5/versions/1.0.0rc1/files/stt_en_quartznet15x5.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2021-05-30 20:44:19 common:654] Instantiating model from pre-trained checkpoint


[NeMo W 2021-05-30 20:44:20 modelPT:133] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2021-05-30 20:44:20 modelPT:140] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data2/voices/train_1k_samp.json

[NeMo I 2021-05-30 20:44:20 features:240] PADDING: 16
[NeMo I 2021-05-30 20:44:20 features:256] STFT using torch
[NeMo I 2021-05-30 20:44:24 modelPT:376] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2021-05-30 20:44:24 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/punctuation_en_distilbert/versions/1.0.0rc1/files/punctuation_en_distilbert.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo
[NeMo I 2021-05-30 20:44:29 common:654] Instantiating model from pre-trained checkpoint


[NeMo W 2021-05-30 20:44:31 modelPT:180] Using /tmp/tmp9i3nvipe/tokenizer.vocab_file instead of tokenizer.vocab_file.


Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.
[NeMo W 2021-05-30 20:44:35 modelPT:133] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    text_file: text_train.txt
    labels_file: labels_train.txt
    shuffle: true
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-05-30 20:44:35 modelPT:140] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    ds_item: null
    text_file: text_dev.txt
    labels_file: labels_dev.txt
    shuffle: false
    num_samples: -1
    batch_size: 64
    
[NeMo W 2021-05-30 20:44:35 modelPT:147] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple_test_data() method and provide a valid configuration file to setup the test data loader(s).
    Test config : 
   

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[NeMo W 2021-05-30 20:44:42 huggingface_utils:95] Config file was not found at distilbert-base-uncased_encoder_config.json. Will attempt to use config_dict or pretrained_model_name.
Some weights of the model checkpoint at distilbert-base-uncased were not used when init

[NeMo I 2021-05-30 20:44:43 modelPT:376] Model PunctuationCapitalizationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/punctuation_en_distilbert/613c4ee780c6fc158f49d3566cbd6636/punctuation_en_distilbert.nemo.
[NeMo I 2021-05-30 20:44:43 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/tts_en_tacotron2/versions/1.0.0rc1/files/tts_en_tacotron2.nemo to /root/.cache/torch/NeMo/NeMo_1.0.0rc1/tts_en_tacotron2/47c6e86fa5d7db74b7855a1af0e55882/tts_en_tacotron2.nemo
[NeMo I 2021-05-30 20:44:46 common:654] Instantiating model from pre-trained checkpoint
[NeMo I 2021-05-30 20:44:47 features:240] PADDING: 16
[NeMo I 2021-05-30 20:44:47 features:249] STFT using conv
[NeMo I 2021-05-30 20:44:48 modelPT:376] Model Tacotron2Model was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/tts_en_tacotron2/47c6e86fa5d7db74b7855a1af0e55882/tts_en_tacotron2.nemo.
[NeMo I 2021-05-30 20:44:48 cloud:66] Downloading from: https://api.ngc.nvidi

[NeMo W 2021-05-30 20:44:58 modelPT:133] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_train.json
      max_duration: null
      min_duration: 0.1
      n_segments: 16000
      trim: false
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 12
      num_workers: 4
    
[NeMo W 2021-05-30 20:44:58 modelPT:140] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.AudioDataset
      manifest_filepath: /raid/LJSpeech/nvidia_ljspeech_val.json
      max_duration: null
      min_duration: 0.1
      n_segments: -1
      

[NeMo I 2021-05-30 20:44:58 features:240] PADDING: 16
[NeMo I 2021-05-30 20:44:58 features:249] STFT using conv
[NeMo I 2021-05-30 20:45:01 modelPT:376] Model WaveGlowModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.0.0rc1/tts_waveglow_88m/90124d3b37eb54a97feac70c86094ff2/tts_waveglow_88m.nemo.


In [5]:
# Convert our audio sample to text
files = [Audio_sample]
raw_text = ''
text = ''
for fname, transcription in zip(files, quartznet.transcribe(paths2audio_files=files)):
  raw_text = transcription

# Add capitalization and punctuation
res = punctuation.add_punctuation_capitalization(queries=[raw_text])
text = res[0]
print(f'\nRaw recognized text: {raw_text}. \nText with capitalization and punctuation: {text}')

[NeMo W 2021-05-30 20:45:09 patch_utils:50] torch.stft() signature has been updated for PyTorch 1.7+
    Please update PyTorch to remain compatible with later versions of NeMo.
[NeMo W 2021-05-30 20:45:09 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
      normalized, onesided, return_complex)
    



[NeMo I 2021-05-30 20:45:09 punctuation_capitalization_model:385] Using batch size 1 for inference
[NeMo I 2021-05-30 20:45:09 punctuation_capitalization_dataset:134] Max length: 27
[NeMo I 2021-05-30 20:45:09 data_preprocessing:297] Some stats of the lengths of the sequences:
[NeMo I 2021-05-30 20:45:09 data_preprocessing:303] Min: 27 |                  Max: 27 |                  Mean: 27.0 |                  Median: 27.0
[NeMo I 2021-05-30 20:45:09 data_preprocessing:305] 75 percentile: 27.00
[NeMo I 2021-05-30 20:45:09 data_preprocessing:306] 99 percentile: 27.00
[NeMo I 2021-05-30 20:45:09 punctuation_capitalization_dataset:165] 0 are longer than 27
[NeMo I 2021-05-30 20:45:09 punctuation_capitalization_dataset:168] *** Example ***
[NeMo I 2021-05-30 20:45:09 punctuation_capitalization_dataset:169] i: 0
[NeMo I 2021-05-30 20:45:09 punctuation_capitalization_dataset:170] subtokens: [CLS] well i don ' t wish to see it any more observed phoebe turning away her eyes it is certainly ve

In [6]:

# A helper function which combines Tacotron2 and WaveGlow to go directly from 
# text to audio
def text_to_audio(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').numpy()

In [7]:

# This is our original audio sample
IPython.display.Audio(Audio_sample)

In [8]:
# This is what was recognized by the ASR model
print(raw_text)

well i don't wish to see it any more observed phoebe turning away her eyes it is certainly very like the old portrait


In [9]:
# This is how punctuation model changed it
print(text)

Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very like the old portrait.


In [10]:
# Without punctuation
IPython.display.Audio(text_to_audio(raw_text), rate=22050)

In [11]:
# Final result - with punctuation
IPython.display.Audio(text_to_audio(text), rate=22050)